<a href="https://colab.research.google.com/github/senthil66/WikiFiles/blob/main/Copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import math
import bs4 as bs
import urllib.request
import re
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import tokenize
from operator import itemgetter

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
# word_tokenize accepts
# a string as an input, not a file.
stop_words = set(stopwords.words('english'))

In [8]:
search_keywords=[]
scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Sachin_Tendulkar')
article = scrapped_data .read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

print(article_text)   


Sachin Ramesh Tendulkar (/ˌsʌtʃɪn tɛnˈduːlkər/ (listen); pronounced [sət͡ʃin t̪eːɳɖulkəɾ]; born 24 April 1973) is a former international cricketer of India who served as captain of the Indian national team. He is regarded as one of the greatest batsmen in the history of cricket.[6] He is the highest run scorer of all time in international cricket, and the only player to have scored one hundred international centuries, the first batsman to score a double century in a One Day International (ODI), the holder of the record for the most runs in both Test and ODI cricket, and the only player to complete more than 30,000 runs in international cricket.[7] In 2013, he was the only Indian cricketer included in an all-time Test World XI named to mark the 150th anniversary of Wisden Cricketers' Almanack.[8][9][10] He is affectionately known as "Little Master" or "Master Blaster".[11][12][13][14]
Tendulkar took up cricket at the age of eleven, made his Test debut on 15 November 1989 against Pakist

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

sentence_list = nltk.sent_tokenize(article_text)
#print(sentence_list)

stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}
for word in nltk.word_tokenize(formatted_article_text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
            
maximum_frequncy = max(word_frequencies.values())

In [11]:
for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 20:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

#print(sentence_scores)

In [12]:
import heapq
summary_sentences = heapq.nlargest(50, sentence_scores, key=sentence_scores.get)

doc = ' '.join(summary_sentences)
#print(summary_sentences, sep = "\n")

#print(type(summary_sentences))
#for s in summary_sentences:
    #print(s)
    
text_tokens = word_tokenize(doc)

#cword = [word for word in text_tokens if not word in stopwords.words()]

#print(type(doc))

val=1
for sen in summary_sentences:
    #print(val,sen)
    val+=1
    
total_words = doc.split()
total_word_length = len(total_words)
#print(total_word_length)

total_sentences = tokenize.sent_tokenize(doc)
total_sent_len = len(total_sentences)
print(total_sent_len)


50


In [13]:
tf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

# Dividing by total_word_length for each dictionary element
tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
#print(tf_score)

def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))
idf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

# Performing a log and divide
idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

#print(idf_score)

tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
print(tf_idf_score)

{'During': 0.005351604658588435, 'second': 0.0004323960970909426, 'match,': 0.0031174672590652936, 'also': 0.0020595423583031573, 'became': 0.0027952154515238884, 'first': 0.0044186014649528, 'score': 0.00042322437563671307, '15,000': 0.007697430141614327, 'runs': 0.0012696731269101393, 'ODIs': 0.005379241741102141, 'On': 0.005351604658588435, '8': 0.004691650966051783, 'November': 0.005351604658588435, '2011,': 0.005351604658588435, 'Tendulkar': 0.019077174181291123, 'batsman': 0.005351604658588435, 'Test': 0.005231581952864566, 'Cricket': 0.01754865970757616, 'scored': 0.001873716420727418, '326': 0.005351604658588435, '(not': 0.005351604658588435, 'out)': 0.005351604658588435, 'innings': 0.0018315476168060992, 'thousand': 0.005351604658588435, 'tournament': 0.005351604658588435, 'Batting': 0.005351604658588435, 'together,': 0.005351604658588435, '6920': 0.005351604658588435, 'India': 0.007585742058111576, 'partnership': 0.003792871029055788, 'average': 0.004691650966051783, 'fifty':

In [14]:
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result

dic=get_top_n(tf_idf_score, 75)


In [15]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [16]:
from textblob import TextBlob
file1 = open('keyw.txt', 'w')
for v in dic:
    if len(v) > 3:
        wiki = TextBlob(str(v))
        for e,val in wiki.tags:
            if(val == 'NN' or val == 'NNS' or val == 'NNPS' or val == 'NNP'):
                search_keywords.append(e)
    
for s in search_keywords:
    file1.writelines(s+"\n")
file1.close()
 
#print(search_keywords)
file1 = open('qns.txt', 'w')
for sentence in summary_sentences:
        vz=sum(1 for word in search_keywords if word in sentence)
        if  (int(vz))>=2 and(int(vz))<=4:
            file1.write(sentence + "\n")
file1.close()

In [22]:
file1 = open("qns.txt","r+") 
doc=file1.read()
con=doc
file1.close()

file1 = open("keyw.txt","r+") 
keys=file1.read()
file1.close()
print(keys.split())
print("\n")
print(con)

['Tendulkar', 'Cricket', 'Azharuddin', 'World', 'Tendulkar', 'Australia', 'Pakistan', 'Cup', 'India', 'first-class', 'Series', 'West', 'cricket', 'ODIs', 'November', 'batsman', 'thousand', 'tournament', 'fifty', 'reach', 'Guyana', 'total', 'post', 'excess', 'time', 'October', 'debut', 'commentator', 'male', 'double-century', 'season', 'clarification', 'seasons']


Tendulkar scored 326 (not out) in this innings and scored over a thousand runs in the tournament.
Batting together, they scored 6920 runs for India at a partnership average of over fifty runs.
Tendulkar played 664 international cricket matches in total, scoring 34,357 runs.
It enabled India to post a score in excess of 300 runs for the first time in an ODI.
He is the first male cricketer to score a double-century in one-day cricket.
Sachin has scored more than 500 runs in IPL in two different seasons as a captain.
In the 2002 series in the West Indies, Tendulkar started well, scoring 79 in the first Test.
The Indian tail coll